<a href="https://colab.research.google.com/github/CantoneseCounsellorChatbot/CantoneseChatbot/blob/main/chatbot_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install simpletransformers
!pip install pytorch_pretrained_bert
# !pip install sqlite3
!git clone https://github.com/CantoneseCounsellorChatbot/CantoneseChatbot.git
# download question retrieval model
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv" -O pretrain-model.zip
!rm -rf /tmp/cookies.txt
! unzip -o -d CantoneseChatbot/ pretrain-model.zip

fatal: destination path 'CantoneseChatbot' already exists and is not an empty directory.
--2021-05-31 13:12:53--  https://docs.google.com/uc?export=download&confirm=K0On&id=1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv
Resolving docs.google.com (docs.google.com)... 74.125.203.113, 74.125.203.100, 74.125.203.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.203.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-c0-docs.googleusercontent.com/docs/securesc/92d7eieb37pd4v71vuusctahp7fat46g/e7g7if9chrpnbv77nullnbflre20sab8/1622466750000/16330851532538606170/04487718260056646995Z/1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv?e=download [following]
--2021-05-31 13:12:54--  https://doc-04-c0-docs.googleusercontent.com/docs/securesc/92d7eieb37pd4v71vuusctahp7fat46g/e7g7if9chrpnbv77nullnbflre20sab8/1622466750000/16330851532538606170/04487718260056646995Z/1r5GRL51-DzoFrl3ba01HZqlPqmVu9cGv?e=download
Resolving doc-04-c0-docs.googleusercontent.com (d

In [2]:
import sys
# from google.colab import drive
# drive.mount('/content/drive')
sys.path.append('/content/CantoneseChatbot/')
from datetime import time
import time
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from tqdm import tqdm
from simpletransformers.classification import ClassificationModel
from transformers import BertTokenizer, BertForSequenceClassification
# import sqlite3
import re
from models import *


In [3]:
# The default response order is advice-->question-->restatement-->bertsum, you can change the order by changing the parameter b
# The threshold of "advice","question"and"restatement" ranges from 0 to 1.1
# The threshold of "bertsum" ranges from -10 to 0
chatbot_params={"advice":{"Threshold":1.0,"order":1},
                "question":{"Threshold":1.0,"order":2},
                "restatement":{"Threshold":1.0,"order":3},
                "bertsum":{"Threshold":-2,"order":4},
                "general":{"Threshold":None,"order":5},
                "mode":"debug"
                }

In [4]:
chatbot(chatbot_params)

input:唉，我最近覺得好唔開心好壓抑呀


  0%|          | 0/4567 [00:00<?, ?it/s]

  0%|          | 0/571 [00:00<?, ?it/s]

chatbot: 最近係咪有啲事令你覺得唔開心呀？不如你講嚟聽下吖？有我喺度陪你㗎。
score:0.998046875


  0%|          | 0/4432 [00:00<?, ?it/s]

  0%|          | 0/554 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# BertSum, still have some bug
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import argparse
import logging
import os
from tqdm import tqdm, trange
import json

from preprocess import LCSTSProcessor, convert_examples_to_features, create_dataset
from model import BertAbsSum
from pytorch_pretrained_bert.tokenization import BertTokenizer
from utils import rouge
from utils import convert_to_unicode
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class InputExample(object):
    """A single training/test example."""

    def __init__(self, guid, src, tgt=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            src: string. The untokenized text of the target sequence.
            tgt: (Optional) string. The untokenized text of the target.
        """
        self.guid = guid
        self.src = src
        self.tgt = tgt

BATCH_SIZE = 1

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument("--model_path",
                    default=None,
                    type=str,
                    required=True,
                    help="The path to trained model.")
parser.add_argument("--config_path",
                    default=None,
                    type=str,
                    required=True,
                    help="The path to config file.")                    
# parser.add_argument("--eval_path",
#                     default=None,
#                     type=str,
#                     required=True,
#                     help="The path to the evaluation data. Should end with .tsv.")
parser.add_argument("--bert_model", 
                    default=None, 
                    type=str, 
                    required=True,
                    help="Bert pre-trained model selected in the list: bert-base-uncased, "
                    "bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, "
                    "bert-base-multilingual-cased, bert-base-chinese.")
# parser.add_argument("--result_path",
#                     default=None,
#                     type=str,
#                     required=True,
#                     help="The path where you save your results.")                    
parser.add_argument("--max_src_len",
                    default=130,
                    type=int,
                    help="Max sequence length for source text. Sequences will be truncated or padded to this length")
parser.add_argument("--max_tgt_len",
                    default=30,
                    type=int,
                    help="Max sequence length for target text. Sequences will be truncated or padded to this length")


if __name__ == "__main__":
    # args = parser.parse_args()
    args = parser.parse_args(args=['--model_path','/content/drive/MyDrive/chatbot/pretrain-model/output_BTS_re+pr05190/model/BertAbsSum_14.bin',
                                   '--config_path','/content/drive/MyDrive/chatbot/pretrain-model/output_BTS_re+pr05190/model/config.json',
                                   '--bert_model','/content/drive/MyDrive/chatbot/BertSum/pretrained_model/bert-base-chinese'])


    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device=torch.device("cpu")
    with open(args.config_path, 'r') as f:
        config = json.load(f)
    model = BertAbsSum(args.bert_model, config['decoder_config'], device)
    model.load_state_dict(torch.load(args.model_path))
    model.to(device)

    processor = LCSTSProcessor()
    tokenizer = BertTokenizer.from_pretrained(os.path.join(args.bert_model, 'vocab.txt'))
    # test_examples = processor.get_examples(args.eval_path)
    post=input("input:")
    test_examples = [InputExample(guid=0,src=convert_to_unicode(post),tgt=convert_to_unicode(post)),InputExample(guid=1,src=convert_to_unicode(post),tgt=convert_to_unicode(post))]
    test_features = convert_examples_to_features(test_examples, args.max_src_len, args.max_tgt_len, tokenizer)
    test_data = create_dataset(test_features)

    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE, drop_last=True)

    model.eval()
    # if not os.path.exists(args.result_path):
    #     os.mkdir(args.result_path)
    # f_log = open(os.path.join(args.result_path, 'log.txt'), 'w', encoding='utf-8')

    hyp_list = []
    ref_list = []
    for batch in tqdm(test_dataloader, desc="Iteration"):
        batch = tuple(t.to(device) for t in batch)
        pred, beam_score = model.beam_decode(batch[0], batch[1], 3, 3)
        src, tgt = batch[0], batch[2]
        for i in range(BATCH_SIZE):
            sample_src = "".join(tokenizer.convert_ids_to_tokens(src[i].cpu().numpy())).split('[CLS]')[1].split('[SEP]')[0] + '\n'
            sample_tgt = "".join(tokenizer.convert_ids_to_tokens(tgt[i].cpu().numpy())).split('[CLS]')[1].split('[SEP]')[0] + '\n'
            sample_pred = "".join(tokenizer.convert_ids_to_tokens(pred[i][0])).split('[SEP]')[0] + '\n'
            sample_pred_2 = "".join(tokenizer.convert_ids_to_tokens(pred[i][1])).split('[SEP]')[0] + '\n'
            sample_pred_3 = "".join(tokenizer.convert_ids_to_tokens(pred[i][2])).split('[SEP]')[0] + '\n'

            print('Hypothesis: ' + sample_pred)



            print('score: ' + str(beam_score[i]))
            
            # f_hyp.write(sample_pred)
            # f_ref.write(sample_tgt)


        
